<a href="https://colab.research.google.com/github/dmarx/advent-of-code_2021/blob/main/%5BAoC_21%5D_Day3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Day 3: Binary Diagnostic

## Part One

The submarine has been making some odd creaking noises, so you ask it to produce a diagnostic report just in case.

The diagnostic report (your puzzle input) consists of a list of binary numbers which, when decoded properly, can tell you many useful things about the conditions of the submarine. The first parameter to check is the power consumption.

You need to use the binary numbers in the diagnostic report to generate two new binary numbers (called the gamma rate and the epsilon rate). The power consumption can then be found by multiplying the gamma rate by the epsilon rate.

Each bit in the gamma rate can be determined by finding the most common bit in the corresponding position of all numbers in the diagnostic report. For example, given the following diagnostic report:

In [3]:
test_input = """
00100
11110
10110
10111
10101
01111
00111
11100
10000
11001
00010
01010
""".strip()

Considering only the first bit of each number, there are five 0 bits and seven 1 bits. Since the most common bit is 1, the first bit of the gamma rate is 1.

The most common second bit of the numbers in the diagnostic report is 0, so the second bit of the gamma rate is 0.

The most common value of the third, fourth, and fifth bits are 1, 1, and 0, respectively, and so the final three bits of the gamma rate are 110.

So, the gamma rate is the binary number 10110, or 22 in decimal.

The epsilon rate is calculated in a similar way; rather than use the most common bit, the least common bit from each position is used. So, the epsilon rate is 01001, or 9 in decimal. Multiplying the gamma rate (22) by the epsilon rate (9) produces the power consumption, 198.

Use the binary numbers in your diagnostic report to calculate the gamma rate and epsilon rate, then multiply them together. What is the power consumption of the submarine? (Be sure to represent your answer in decimal, not binary.)

In [40]:
import torch
import numpy as np
def bits2tensor(in_str: str) -> torch.Tensor:
  return torch.ByteTensor([[int(tok) for tok in row] for row in in_str.split('\n')])
  #return np.array([[int(tok) for tok in row] for row in in_str.split('\n')])

t_test = bits2tensor(test_input)

def calc_gamma_(diagnostics_tensor: torch.ByteTensor):
  n = diagnostics_tensor.shape[0]
  return 1*(diagnostics_tensor.sum(dim=0) > n//2) # does this still work if n is odd?
  
def calc_gamma_rate(diagnostics_tensor: torch.ByteTensor):
  gamma_binary = calc_gamma_(diagnostics_tensor)
  return binaryVect2Decimal(gamma_binary)

def calc_eps_(diagnostics_tensor: torch.ByteTensor):
  gamma_binary = calc_gamma_(diagnostics_tensor)
  return gamma_binary < 1

def calc_eps_rate(diagnostics_tensor: torch.ByteTensor):
  eps_binary = calc_eps_(diagnostics_tensor)
  return binaryVect2Decimal(eps_binary)

def binaryVect2Decimal(vect: torch.Tensor) -> int:
  reversed = vect.numpy()[::-1]
  outv = 0
  for idx, val in enumerate(reversed):
    if val:
      outv += 2**(idx)
  return outv

assert calc_gamma_rate(t_test) == 22
assert calc_eps_rate(t_test) == 9

In [41]:
# part one solution
# 'data' defined at bottom
t_data = bits2tensor(data)
calc_gamma_rate(t_data) * calc_eps_rate(t_data)

3923414

## Part Two

Next, you should verify the life support rating, which can be determined by multiplying the oxygen generator rating by the CO2 scrubber rating.

Both the oxygen generator rating and the CO2 scrubber rating are values that can be found in your diagnostic report - finding them is the tricky part. Both values are located using a similar process that involves filtering out values until only one remains. Before searching for either rating value, start with the full list of binary numbers from your diagnostic report and consider just the first bit of those numbers. Then:

- Keep only numbers selected by the bit criteria for the type of rating value for which you are searching. Discard numbers which do not match the bit criteria.
- If you only have one number left, stop; this is the rating value for which you are searching.
- Otherwise, repeat the process, considering the next bit to the right.
The bit criteria depends on which type of rating value you want to find:

- To find oxygen generator rating, determine the most common value (0 or 1) in the current bit position, and keep only numbers with that bit in that position. If 0 and 1 are equally common, keep values with a 1 in the position being considered.
- To find CO2 scrubber rating, determine the least common value (0 or 1) in the current bit position, and keep only numbers with that bit in that position. If 0 and 1 are equally common, keep values with a 0 in the position being considered.

For example, to determine the oxygen generator rating value using the same example diagnostic report from above:

- Start with all 12 numbers and consider only the first bit of each number. There are more 1 bits (7) than 0 bits (5), so keep only the 7 numbers with a 1 in the first position: 11110, 10110, 10111, 10101, 11100, 10000, and 11001.
- Then, consider the second bit of the 7 remaining numbers: there are more 0 bits (4) than 1 bits (3), so keep only the 4 numbers with a 0 in the second position: 10110, 10111, 10101, and 10000.
In the third position, three of the four numbers have a 1, so keep those three: 10110, 10111, and 10101.
- In the fourth position, two of the three numbers have a 1, so keep those two: 10110 and 10111.
- In the fifth position, there are an equal number of 0 bits and 1 bits (one each). So, to find the oxygen generator rating, keep the number with a 1 in that position: 10111.
- As there is only one number left, stop; the oxygen generator rating is 10111, or 23 in decimal.

Then, to determine the CO2 scrubber rating value from the same example above:

- Start again with all 12 numbers and consider only the first bit of each number. There are fewer 0 bits (5) than 1 bits (7), so keep only the 5 numbers with a 0 in the first position: 00100, 01111, 00111, 00010, and 01010.
- Then, consider the second bit of the 5 remaining numbers: there are fewer 1 bits (2) than 0 bits (3), so keep only the 2 numbers with a 1 in the second position: 01111 and 01010.
- In the third position, there are an equal number of 0 bits and 1 bits (one each). So, to find the CO2 scrubber rating, keep the number with a 0 in that position: 01010.
- As there is only one number left, stop; the CO2 scrubber rating is 01010, or 10 in decimal.
- Finally, to find the life support rating, multiply the oxygen generator rating (23) by the CO2 scrubber rating (10) to get 230.

Use the binary numbers in your diagnostic report to calculate the oxygen generator rating and CO2 scrubber rating, then multiply them together. What is the life support rating of the submarine? (Be sure to represent your answer in decimal, not binary.)

In [39]:
# Pretty sure this is equivalent to finding the min and max values. 
for rec in data.strip().split():
  break
report_values = [int(rec,2) for rec in data.split()]
min(report_values) * max(report_values) # 4095 - incorrect, too low

# Ok, yeah. there's a difference between this bitwise min/max and the global min/max. Makes sense.

# Maybe this could be solved by encoding the report as a binary tree and 
# following the paths whose branches have the most(O2)/fewest(CO2) values?

4095

In [73]:
!pip install loguru

     |████████████████████████████████| 57 kB 2.4 MB/s 


In [86]:
from loguru import logger

def _life_support_iter(candidates: torch.Tensor, position: int, argmax: bool=True) -> torch.Tensor:
  n = candidates.shape[0] / 2
  vals = candidates[:,position]
  score = vals.sum()
  if argmax:
    target = score >= n
  else:
    target = score < n
  logger.debug((score, n, target))
  test = vals == target

  return candidates[test,:]

def _life_support_filter(report: torch.Tensor, argmax=True) -> torch.Tensor:
  candidates = report
  for i in range(report.shape[1]):
    candidates = _life_support_iter(candidates=candidates, position=i, argmax=argmax)
    logger.debug(i)
    logger.debug(candidates)
    if candidates.shape[0] == 1:
      break
  return binaryVect2Decimal(candidates.squeeze(0))
  #return candidates

def get_oxygen_rating(report: torch.Tensor):
  return _life_support_filter(report, argmax=True)

def get_carbon_rating(report: torch.Tensor):
  return _life_support_filter(report, argmax=False)

logger.remove() # remove default handler to suppress debugging messages
assert get_oxygen_rating(t_test) == 23
assert get_carbon_rating(t_test) == 10

In [87]:
# part two solution

get_oxygen_rating(t_data) * get_carbon_rating(t_data)

5852595

In [34]:
data = """
000110011011
010011010000
111101010001
111101001000
000010001110
101111111010
110111011000
111010011100
101101100011
100011010000
111010111100
110011000110
000100110100
010101111101
000100111101
100110001111
000001000101
000101100011
110001001101
110100000100
000111001011
111110011011
110110010000
011001111110
010100101010
000100111100
011000110110
101010110011
101110000111
111000111011
001000000110
010100000000
010000010000
100100100000
110010011100
110000001011
001010011111
001010100101
100100101111
101110001000
011011010101
000111100000
100111101011
000010110000
000100001110
111110000000
101100111111
000010010111
100001011111
010111011100
110101011011
001010010010
111100101001
101100101100
001011001110
001001111001
010010101001
011000000100
000011001110
110100111000
110011101110
100011111111
010111001010
110100000011
101101010010
100101000100
011010011001
100110000110
011111110000
010110110110
111100111000
100011111001
010110011011
000001001001
000111011010
111110001101
001011011100
011110100101
000000111000
100110011010
001110011000
110110110010
011111000101
011100101100
010001111111
001011011000
011010100010
011101001001
001101100000
010011010100
111001000011
000000110100
011010101100
101001110000
010000001011
001011101101
010000101010
100000010001
001000101101
100101100111
110101100011
011110010111
010011000010
100111111000
001010111100
010010111100
011100111100
100111011111
000100001011
100000010111
100111001101
100100111010
011100001001
000110110001
001100100001
111011101101
000001010100
010000100011
111000111110
111011110001
010111000111
011000011001
010101110001
111000101001
111010000000
100010011111
111000111000
010011011111
101110011101
000110000001
010010011011
100101101000
110101111011
100010101001
101010011010
000100111000
011011101001
100100000100
000010110100
100100000111
101110000011
000100110010
000110100110
110111111010
100000110101
100111000000
111111001001
110100111101
100000000110
010101000011
000101101010
000101111001
101100010000
101111001010
010011100110
011001111100
011001101010
111001110001
011100011101
111011111110
100010010100
111000101010
100000001011
100111110001
110110100111
001000111001
010101000000
001010001011
001010001100
011110111011
011101111111
011100101101
111000111010
101101011100
111010001000
010011011001
100011100101
100001110110
110010001001
101101001011
100100100111
111011110101
011001000100
110010110010
011011111000
111011001000
011011000010
010111000110
001001110011
010001011000
111001100110
001001011000
010101010100
001001001111
001100110010
100001111000
100001000101
001000100111
010110100001
000011101001
100011110110
001000001101
001000010100
001010101000
100001111101
100101110101
100011000001
010001110110
011011010111
011001010110
010101000110
110010100010
111011001111
000111011111
110001110100
000100110101
101001110110
110111000011
010001000000
111000000110
001100011110
101101100001
100000101111
011010010001
011001001011
111001100001
110100001100
111001100000
011011001000
010011011000
000100000100
010000001100
010010001110
011001100101
001011100111
010111010111
110010111101
011011011001
101011100110
001001001001
010000111001
000001011101
001111110010
011100010010
001010100100
111111101011
010100001011
100110000111
111110101110
110111110110
111010100100
111010010010
011100000110
011100100010
010001101111
001100000100
010101001111
010101000100
111111010010
110010100100
011110110010
110100001011
010010100110
001001110111
111101010111
101010000110
111000100110
110000101110
010100010011
000111011100
001010111000
110000011010
000111000100
011011010000
101101000110
111100110111
000001001000
010011001100
111001001100
111101111110
100100000001
111101111011
001101100100
010100011010
101000111000
011100100101
111001101011
101010110010
011010000100
101101110001
101110001101
000000101001
100001010010
000000111101
000100100100
110010110011
010001001011
111010000101
111100100011
001110110101
000011100001
110011010100
001011101001
110100101101
001111110011
001001001110
100011010100
001001111010
111101001001
100100000000
111000010111
100010011100
101000110011
000010101011
101100111000
011000010110
101101100100
101010100110
010110000010
000110001000
010111010110
110001000101
111010011110
011101100101
101011011111
011101100100
111111101100
001111001000
001100101011
110000000110
001000001110
111001001001
001111100111
001110000101
001001011110
111001110100
111100111011
101000100001
110101011101
001100010100
100111111101
101001000111
000111001001
111010000001
100101110110
001110001110
111001110000
111000110010
011110110001
001101100001
111010010111
111011100100
011000111101
111111010011
010110111110
011010111101
001110000110
100000100010
010100110001
111101101010
000101010000
100010001001
000000000001
011011000100
111000011100
110000010100
001101011011
110110010001
011100001110
110010010011
110000010111
110000100011
101100000101
110011110011
000101101000
111011010001
101111000010
001011010101
100110010001
001010001101
011010110110
100110010111
011101010011
101101101111
011011000001
101100011111
100110010100
011001101011
010001010011
111001000110
001000111101
001101000100
101000000001
100011111000
111000011110
010100000110
100110110101
110111111000
101010110100
011100011110
101111101110
000110101111
001001100101
111100001000
011011100100
110011111110
101110110101
110111101000
000001111000
001011000001
001110101010
000100010100
011011100010
100111111110
001111110001
001001110101
000001011110
110010010001
110100011011
101101010110
110111000100
100100110011
111001101110
100000000011
011100001010
011001011001
101110000000
101111000011
010010111110
110000101000
000001000111
100000101110
110101011100
100011000011
111000111100
011000001101
000011001001
000101000100
010101001100
011110010001
000011001000
000011011111
001010110011
011001011000
010111111011
100100110010
011100111010
110111100111
011110111101
001010111111
110111010000
000100000011
111111110110
000011010011
010111011110
111001111101
111111010110
000111000110
000100100000
101011000000
000101110101
111001000100
101101100010
100001001110
011011001011
111001011011
100010100010
001001010000
011011000110
101100011110
111000110000
011000111001
100101000010
010110101011
101010001000
110001001111
001110101011
100100001101
000010001101
010011111111
100010010001
010010001111
100011011111
101100001011
101101101110
111100000110
000111111001
010111000000
011110011011
101101111101
001000000010
110001001110
101110101111
110001011011
000100101010
110111111111
110111000110
011101101000
111101110001
100110100010
011100010111
001001111101
101010010000
010110010001
110011100101
000010101101
111101101111
111000100100
011000111000
111101000111
111111111000
001010011000
101011011001
111000000000
111010100011
100111110100
110111101100
001100111011
100011101000
000100101111
100001011110
100110011000
010101010001
110111100010
110000110011
101101011111
110110100010
111101001100
010010010101
101010101100
001111111001
011111100001
101101110101
101011001011
111010010000
001111000000
101001110100
110110110101
000110000101
000111000111
011000110101
100001010101
100101001110
001101101111
010111100001
101000011111
011101110110
101010101111
001110110011
111000010011
001110101100
011110110101
001000011101
011001110000
001010010101
000101010110
001100100000
111101011111
101000010110
010000000000
101011010110
011010000111
110000111000
100010110110
101100100000
110011000011
010100111000
001000001000
000100111001
000110000100
110100001000
011001010111
111110101101
001011011101
101000101110
000000011100
100000110001
001110100101
111100000001
111011101100
001100011010
101001100110
010111011101
000111110001
010110000000
000001010000
011001001101
110011100011
111111111111
010110011001
100011001111
010111101010
110010111100
111000001011
101110010000
100000100001
100001110100
000011110110
101000011010
111110010111
111010011011
111001101100
000000011110
111101110010
101010001010
010001010001
110010100011
000010101010
100011111010
001000000101
001110010011
000000110111
010100010000
001111100101
001000010101
110010011010
100100100001
001011101110
110110010111
000110011111
000001111001
101001101100
010111001011
001001001000
111101000110
100000010000
001011000111
110010001100
100000000100
000010101111
111100010001
001100001100
101001101110
011010100011
000101100001
011101011010
011110110011
010111001001
101111010101
000100000001
100001110001
011101001111
100010000111
110101010100
111000001100
010001001010
101000010010
001101001110
101010011110
010101100111
011001000011
000110111111
101110100110
001001101110
010100111001
010010001100
100100010011
000011110111
101100100001
000010111101
011001010100
110000100111
110011111000
101100110000
001111001001
110001111111
001101011110
010110111111
011111100011
011001001000
101100000010
011110011111
101101100111
100010001111
000001100001
100100001110
001000110111
010001001101
110111111100
101110100100
101010010010
001100100011
100101110001
110111100001
010100101101
110000000000
111110001010
010111100110
100101000001
000101111100
110000000001
110101011110
001010010100
000111110011
011101000001
001100110100
100010101100
110100000000
100010100011
111010100111
010110001110
000001101010
010011000001
111000100101
000001010110
000000110011
001111111101
101111110001
100100111111
110101101001
111011100110
100110011101
110001100111
010010010001
101001100100
010110000001
101110010001
101000111110
101010100000
000011100010
010010110001
101100000110
111010010110
100101001100
000101010011
101010111111
101100010010
101111100010
101101101100
000110010100
111111110011
100111010100
000100000010
101000000000
100101010011
100111101110
100111110000
100010111001
001010011100
010111110000
001101001000
100011010110
101010101000
001111111111
011111001111
001000110100
000010100100
010001001110
000011011110
011010001010
111001010010
001111101001
010101100110
000110101001
100011011001
000111000010
010111001100
110111011001
000010100110
100011001100
110110101100
010001100110
011111001110
101110010100
001011000110
101101111011
010011101010
110001110111
100000010101
001110001010
111000000111
110011010101
101010010111
001011000011
101110010010
011101100110
101011011011
111000110001
111101011100
100001000010
100001100000
010011010011
101010101010
111011110010
100010111000
101110011010
011001001001
101000101010
000001000100
010110011111
001110010101
111100101011
101011110111
100000111001
111101100010
100111000100
100001001000
101100011011
010011000100
110110101101
010100011001
100000111010
101100000000
110110010101
101001000000
000100000000
010101001110
001011001111
010010011101
010110001111
000000001111
110110010010
000011010000
101000110001
110001101110
001011111000
011111101011
110000001000
010100111111
100001100110
011111010101
111100101000
101101110111
001111000100
010000000001
111010101000
001111011011
001001100110
101000001100
100110010011
100001011010
110101001010
010011000111
101101011011
110110000001
010011111001
000010011101
001101000111
011110010010
001100101101
110100011110
001000110000
101101000001
101000101001
101100110001
010110101110
110010011101
001111011100
011000010111
000011100000
100010110000
001010110001
001011111110
001000000011
011110110000
100010111110
100111101010
100111010010
010010110010
101110110110
111011011111
011111011100
101111110100
110011101011
110001000001
101101010001
011000011111
010000110111
001111101010
011100011111
001000100000
101110001001
101000001110
000010000010
111010100010
001111011000
000010111000
101101100110
111100011001
010100011110
101101010000
001000100110
011010110010
001111101111
010111101000
010110011010
001000010010
110101010111
101111110010
111110111101
100100111110
000101000110
000100010011
010000111111
110111011111
111100001111
011011110100
000101011000
111001011010
000101101111
110000101111
011101001101
000001010011
000001001111
000111010000
001000011110
111111001101
110100011101
110110111111
111000101000
000000110110
000111101010
010001100001
011111100111
101100000100
101001001101
011101101101
110010001010
101100110111
011001111011
111100010010
010111000010
000001000010
111110101100
111011101000
111111110111
111001111110
010011010101
010101010000
000101011011
110011010001
001001010011
011111111101
101001100001
001010010110
100001010111
110011100110
110000001101
011110111010
100011101110
101101001010
101110000110
011011100101
101111010100
001011101011
110001101011
100011011010
111001010110
001110101000
111110000111
011011011111
100011000110
011000101101
100011001110
001101101101
110111101011
110000000101
010010000110
110000111100
000000001110
110110011001
010111000001
111000010110
000101001111
011001111010
101100110010
001111110000
011100110000
010110111011
100110001101
111110111111
010001100101
101011110110
110000100001
000001100110
000000100011
101110101010
110100011001
101111111111
111001101000
111100011101
100010001000
011110101110
111111010001
110010110000
001111000101
110110001010
000101000010
011011111001
100101000111
010110000111
000101001010
110010100110
110000110001
110101011001
010111001000
111111001110
000110110101
001100010110
011010100110
""".strip()